<a href="https://colab.research.google.com/github/fayshaw/data_preprocessing/blob/main/livwell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preproccesing Best Practices

## LivWell Dataset: Women and their Well-being for 52 Countries

###<a href="https://www.womenindata.org/">Women in Data Boston</a> and <a href="https://www.meetup.com/pyladies-boston/">PyLadies Boston</a>
#### <a href="https://www.linkedin.com/in/fayshaw/">Fay Shaw</a>
August 21, 2025

Together, we will explore the LivWell dataset from the Belmin et al's 2022 Nature paper <a href=" https://www.nature.com/articles/s41597-022-01824-2"> LivWell: a sub-national Dataset on the Living Conditions of Women and their Well-being for 52 Countries</a>. The authors constructed a longitudinal dataset using Demographic and Health Surveys (DHS), GDP data, and climate data for subnational regions for 1990 - 2019.

*Figure 1: Flowchart representing the data processing steps to obtain LivWell. Orange: input data; green: indicators based on DHS data; blue: indicators based on gridded data; white: validation data.*

<img src="https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41597-022-01824-2/MediaObjects/41597_2022_1824_Fig1_HTML.png" width="600">


In this notebook, we will look at some of the DHS STAT data compare it to their data output.

🚩 <a href="https://github.com/fayshaw/data_preprocessing">Github repository</a>



# 🎯 Goals for this session
1. Learn the context of the data.
2. Transform raw dataset to compare to LivWell
3. Visualize data.


# Files
1. Load LivWell dataset using urls: `livwell.csv` and `indicators.csv`
2. Load DHS STAT raw data: `STATcompilerExport_decision_power.csv`
3. Load global wealth indicators: `GDL-Mean-International-Wealth-Index-(IWI)-score-of-region-data.csv`

Both the DHS STAT and Global Mean data are found in the authors' <a href="https://gitlab.pik-potsdam.de/belmin/livwelldata-paper/-/tree/main/analysis/data/raw_data/validation_data?ref_type=heads"> validation data folder</a> on gitlab.

# 💡 1. Learn the context by exploring the data

## Read files
The first thing to do is to look at the the LivWell data.  We can open it in Excel and read in the file using pandas.

In [ ]:
import pandas as pd
livwell_df = pd.read_csv('https://zenodo.org/records/7277104/files/livwell.csv')

### Explore the data

<img src="https://scentla.com/wp-content/uploads/2025/02/Efficiently-Create-and-Fill-Pandas-DataFrames-in-Python-1024x399.jpg" width=600>

Figure from https://datagy.io/pandas-drop-index-column

Resources
* <a href="https://realpython.com/pandas-python-explore-dataset/">Real Python dataframe resource</a>
* <a href="https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf">PyData Pandas cheat sheet</a>

DataFrame `df`
* Show `df`
* `df.head()`
* `df.describe()`
* `df.columns`
* `df.unique()`

In [ ]:
livwell_df

In [ ]:
livwell_df.columns

In [ ]:
indicators_df = pd.read_csv("https://zenodo.org/records/7277104/files/indicators.csv")
indicators_df

Look at one column of data using the dataframe `df` and the column name `col`: `df['col']`.  You can use `set()` to find the set of values.

In [ ]:
set(indicators_df['indicator_category'])

In [ ]:
# Find unique countries
livwell_df['country_name'].unique()

In [ ]:
# The length of the set of
len(livwell_df['country_name'].unique())

## Filter to get data for one country


In [ ]:
# Choose one column and check if the value is Armenia.
livwell_df['country_name'] == 'Armenia'

In [ ]:
# Create a new dataframe livwell_armenia for that country
livwell_armenia = livwell_df[livwell_df['country_name'] == 'Armenia']

# Years with survey data
print("years: " , set(livwell_armenia['year']))

# Regions in Armenia
print("regions: ", set(livwell_armenia['region_name_harmonized']))
livwell_armenia.head(12)

## 🔎 More to explore with dataframes
pandas DataFrame: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html
* `df.shape`
* `df.dtypes`
* `df['column'].value_counts()`

# 🚀 2. Transform raw dataset to compare to LivWell

## Read STAT file

Manually upload the file `STATcompilerExport_decision_power.csv`

In [ ]:
from google.colab import files
uploaded = files.upload()

🔎 Notice that there `Unnamed` column names at the top along with NaN rows at the top and bottom

In [ ]:
stat_power = pd.read_csv("STATcompilerExport_decision_power.csv")
stat_power

🤔 How many rows are null?  Can we safely skip them?

* Top 2 rows are null and we want the third row to be for titles.
* Bottom rows are not regional data.

In [ ]:
stat_power.iloc[0].isnull().sum()

In [ ]:
# Check out the tail end.  Rows 714-727 are not regional data.
stat_power.tail(15)

In [ ]:
# Read the file again by skipping the top NaN rows and bottom rows
stat_power = pd.read_csv('STATcompilerExport_decision_power.csv', skiprows=3, skipfooter=14, engine='python')
stat_power

In [ ]:
set(stat_power['Country'])

In [ ]:
# Look at the length of this list
len(set(stat_power['Country']))

## One country example

* Get data for just Armenia
* Make a deep `.copy()` so you are not operating on a view and avoid the <a href="https://realpython.com/pandas-settingwithcopywarning/">`SettingWithCopyWarning`</a>

In [ ]:
stat_armenia_power = stat_power[stat_power['Country'] == 'Armenia'].copy()
stat_armenia_power.head(15)

In [ ]:
stat_armenia_power.columns

## 🦋 Transform data step by step
Find columns that could be in a more useful format.

### Get the survey years
Take out the space between the year and the survey name.

In [ ]:
set(stat_armenia_power['Survey'])

In [ ]:
# Split the survey year text on the space ' ' to get the year.
# Make two new columns 'year_text' and 'source' that appear at the right side.
stat_armenia_power[['year','source']] = stat_armenia_power.loc[:, 'Survey'].str.split(expand=True)
stat_armenia_power.head(15)

In [ ]:
### Rename the year text 2015-16 to 2016.
stat_armenia_power['year'] = stat_armenia_power['year'].replace('2015-16', '2016')
stat_armenia_power.head()

In [ ]:
# Similarly, split the region using by the colin " : "
stat_armenia_power['region'] = stat_armenia_power.loc[:, 'Characteristic'].str.split(" : ").str[1]
stat_armenia_power.head()

### Drop rows that are not regions
📌 Tip: I like to name new dataframes when I've done significant operations like dropping rows.  This avoids errors if I re-run cells.

In [ ]:
# Drop rows that are not regions
stat_armenia_power = stat_armenia_power[~stat_armenia_power['Characteristic'].str.contains('Total')]
stat_armenia_power.head()

In [ ]:
# Rename dataframe
stat_armenia_pow = stat_armenia_power.drop(columns=['Survey', 'Characteristic'])
stat_armenia_pow.head()

In [ ]:
# Rename education columns
rename_DP_cols = {
    'Country' : 'country_name',
    'Wife earns more than husband' : 'DP_earn_more_p',
    'Women who decide themselves how their earnings are used' : 'DP_decide_money_p',
    'Do not own a house [Women]' : 'STAT_not_homeowner'
}

In [ ]:
stat_armenia_DP = stat_armenia_pow.rename(columns=rename_DP_cols)
stat_armenia_DP.head()

### Choose and reorder columns

In [ ]:
# Now it looks similar to the LivWell data
stat_armenia_DP = stat_armenia_DP[['country_name', 'year', 'region', 'DP_earn_more_p', 'DP_decide_money_p', 'STAT_not_homeowner']]
stat_armenia_DP.head()

## 👍  Now this looks similar to our LivWell data set.
* Survey years are numbers (not `2015-2016 DHS`)
* Regions are just names (no `' : '`)
* Renamed columns of interest

## Filter LivWell columns to match STAT columns
### Get LivWell Aremnia decision and power columns

In [ ]:
# These are all of the LivWell (lw) decision and power (DP) columns
lw_all_pow_cols = livwell_armenia.columns[livwell_armenia.columns.str.contains('DP')].to_list()
lw_all_pow_cols

In [ ]:
# Choose columns of interest
lw_DP_cols = ['country_name', 'country_code', 'year', 'region_num_harmonized',
       'region_name_harmonized', 'DP_earn_more_p', 'DP_owns_house_p','DP_decide_money_p']

In [ ]:
# LivWell data for general and power columns
livwell_armenia_DP = livwell_armenia[lw_DP_cols]
livwell_armenia_DP

In [ ]:
# Ckeck our columns for our datasets
print(stat_armenia_DP.columns)
print(livwell_armenia_DP.columns)

## Get data in the same format to merge on year and region.

In [ ]:
# In the LivWell data, the year data is of type int
livwell_armenia_DP['year'].dtype

In [ ]:
# In the STAT data, the year data is of type object
stat_armenia_DP['year'].dtypes

In [ ]:
# Recast the year as an int.  We need to do this to prevent a
# ValueError: You are trying to merge on object and int64 columns for key 'year'
stat_armenia_DP['year'] = stat_armenia_DP['year'].astype(int)
stat_armenia_DP['year'].dtype

## 🧩 Merge the data!

Choose which variables should be the same.  
* On the left (STAT data) those are `country_name`, `region`, and `year`.   
* On the right (LivWell data) those are `country_name`, `region_name_harmonized` and `year`.  
* For the columns that are the same, the new column names will have a suffix (`_stat, `_lw`) to differentiate.

In [ ]:
merged_df = stat_armenia_DP.merge(livwell_armenia_DP, left_on=['country_name', 'region', 'year'],
            right_on=['country_name', 'region_name_harmonized', 'year'],
            suffixes=('_stat', '_lw'))

merged_df.columns

In [ ]:
merged_df

And hopefully they match!   Reorder columns to get a better look.

Note that `STAT_owns_house_p` is the inverse of `STAT_not_homeowner`.

🔎 More to explore
* Can you check to see that `STAT_owns_house_p` and `STAT_not_homeowner` add up to 100%?
* Can you upload other STAT files to compare to LivWell?

In [ ]:
merged_df[['country_name', 'year', 'region', 'DP_earn_more_p_lw', 'DP_earn_more_p_stat',
           'DP_decide_money_p_lw', 'DP_decide_money_p_stat',
           'DP_owns_house_p', 'STAT_not_homeowner']]

## 🌍 Second data source: Global Data Lab Mean International Wealth Index

Find this file to upload: `GDL-Mean-International-Wealth-Index-(IWI)-score-of-region-data.csv`


In [ ]:
uploaded = files.upload()

In [ ]:
gdl = pd.read_csv("GDL-Mean-International-Wealth-Index-(IWI)-score-of-region-data.csv")
gdl

In [ ]:
gdl_armenia = gdl[gdl['Country'] == "Armenia"]

In [ ]:
# Drop the years with NaN values
# The years that are left are similar to our LivWell data
gdl_armenia_years = gdl_armenia.dropna(axis=1)
gdl_armenia_years

In [ ]:
# Keep rows where Region is not 'Total'
gdl_armenia_years = gdl_armenia_years[gdl_armenia_years['Region'] != 'Total']
print(gdl_armenia_years.columns)

# Reorder columns
gdl_armenia_years = gdl_armenia_years[['Country', 'ISO_Code', 'GDLCODE',	'Region', '2000', '2010', '2016']]
gdl_armenia_years

## 🧊 DataFrame melt

This transformation changes our "wide" dataframe to a "long" format.  It returns an "unpivoted" datafrrame.  

We will melt our dataframe and then create a scatter plot.

<img src="https://pandas.pydata.org/pandas-docs/version/0.25.1/_images/reshaping_melt.png" width=800>


Figure from [pandas.pydata.org](https://pandas.pydata.org/pandas-docs/version/0.25.1/user_guide/reshaping.html#reshaping-by-melt)

In [ ]:
# In the melt command, the id_vars are the ones to keep as identifiers
# The var_name year gets added into every row
# The value_name is the International Wealth Index
gdl_armenia_melt = gdl_armenia_years.melt(id_vars=['Country', 'ISO_Code','GDLCODE', 'Region'], var_name='Year', value_name='IWI')
gdl_armenia_melt

### 📅 Include columns that are years by checking to see that they are numerical.

In [ ]:
gdl_year_cols = [x for x in gdl.columns if str.isdigit(x)]
gdl_year_cols

In [ ]:
# Include these columns in the data
gdl_data = gdl[['Country', 'GDLCODE'] + gdl_year_cols]
gdl_data

### Melt the whole GDL data to move years into one column

🔎 More to explore: How can you combine the GDL dataset with the LivWell and STAT datasets?

In [ ]:
gdl_melt = gdl.melt(id_vars=['Country', 'ISO_Code','GDLCODE', 'Region'], var_name='Year', value_name='IWI')
gdl_melt

### Filter the data for plotting
Choose to include only include regional data. Drop data that has the string 'Level' in the Year column

In [ ]:
gdl_region = gdl_melt.drop(columns='Region')
gdl_region = gdl_region[~((gdl_region['Year'] == 'Level') | (gdl_region['IWI'] == 'National'))]
gdl_region

# 📊 3. Data visualization
Plot a subset of countries

In [ ]:
livwell_gdl_subset = ['Armenia', 'Burundi', 'Cambodia', 'Dominican Republic', 'El Salvador',
                      'Fiji', 'Gabon', 'Haiti', 'Tanzania', 'Turkey', 'Yemen', 'Zimbabwe']
livwell_gdl_countries = set(livwell_df['country_name']) & set(gdl_melt['Country'])

print("Countries in LivWell and GDL datasets: ", len(livwell_gdl_countries))
print("Number of subset countries: ", len(livwell_gdl_subset))

### Finding non null data

In [ ]:
#Check if there is data for a year that is not null.
gdl_region[(gdl_region['Year'] == '1992') & (gdl_region['IWI'].notna())]
print(gdl_region.head(10))

## Create a scatter plot of the IWI by year and colored by country.

In [ ]:
import plotly.express as px

gdl_subset = ['Armenia', 'Burundi', 'Cambodia', 'Dominican Republic', 'El Salvador',
              'Fiji', 'Gabon', 'Haiti', 'Tanzania', 'Turkey', 'Yemen', 'Zimbabwe']
gdl_subset_data = gdl_region[gdl_region['Country'].isin(livwell_gdl_subset)]
fig = px.scatter(gdl_subset_data, x="Year", y="IWI", color="Country")
fig.show()

# 🗺️ More to explore

The authors incorporated many  approaches in their work including:

* Analysis in R.  Check out their <a href="https://gitlab.pik-potsdam.de/belmin/livwelldata">LivWell R repository</a>. They linearly interpolated data using the R package `imputeTS`.
* Collapsed categories for <a href="https://gitlab.pik-potsdam.de/belmin/livwelldata-paper/-/blob/main/analysis/data/raw_data/all_labels_cooking_fuel_completed_coal_as_traditional.csv?ref_type=heads">modern and traditional cooking fuel</a>
  * Modern: electricity, liquefied
petroleum gas, natural gas, kerosene and biogas
  * Traditional: biomass (firewood, charcoal, agricultural crops, coal)
  * This could also be described as recoding, label encoding, or feature engineering
* Recoded drinking water quality to low, medium, high quality.  
* <a href="https://gitlab.pik-potsdam.de/belmin/livwelldata-paper/-/blob/main/analysis/data/derived_data/region_harmonization_files/dhs_region_harmonization.csv?ref_type=heads">Geographic data</a>. The authors harmonized variables over time and across countries.  

# 🎉 Takeaways
* Learn the context of the data.
* Before transforming, have a target usage and format in mind.
* You don't need to know all the syntax.  Practice!